<a href="https://colab.research.google.com/github/anujsahani01/technical-assessment/blob/main/Efficient_Trips.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# importing libs
import pandas as pd
from sklearn.cluster import KMeans
import datetime

In [4]:
df = pd. read_excel('/content/Dataset.xlsx')
# removing duplicates
df.drop_duplicates(inplace = True)
# checking presence of null values
if df.isnull().values.any():
  df.dropna(inplace = True).reset_index(drop = True, inplace = True)
# in this problem statement we have no use of time
df['Customer_Location'] = df['Customer_Location'].str.lower()
df['Vehicle_ID'].str.lower()
# Calculate difference
def time_diff(start_time, end_time):
  start_time = datetime.datetime.combine(datetime.date(1, 1, 1), start_time)
  end_time = datetime.datetime.combine(datetime.date(1, 1, 1), end_time)
  diff = end_time - start_time
  return diff
df['TimeDifference'] = df.apply(lambda row: time_diff(row['Delivery_Window_Start'], row['Delivery_Window_End']), axis=1)
df['TimeDifference'] = df['TimeDifference'].apply(lambda td: int(td.total_seconds() / 3600))

df.drop(['Delivery_Window_Start', 'Delivery_Window_End'], axis= 1, inplace = True)
new_df = df.groupby('Vehicle_ID', as_index = False).apply(lambda x: x.sort_values('Distance_to_Location (km)'))

In [5]:
new_df.head(10)

Order_ID Customer_Location  Package_Weight (kg) Vehicle_ID  \
0 20        21        location_u                  2.8         V1   
  41        42       location_pp                  6.9         V1   
  26        27       location_aa                  4.0         V1   
  0          1        location_a                  5.2         V1   
  11        12        location_l                  6.8         V1   
  14        15        location_o                  4.9         V1   
  5          6        location_f                  4.2         V1   
  29        30       location_dd                  5.9         V1   
  17        18        location_r                  6.2         V1   
  38        39       location_mm                  3.4         V1   

      Distance_to_Location (km)  TimeDifference  
0 20                         10               1  
  41                         12               2  
  26                         14               2  
  0                          15               2  
  11                         19               2  
  14                         21               2  
  5                          22               2  
  29                         22               2  
  17                         24               2  
  38                         24               2

In [6]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=4)  # Example: Cluster locations into 3 regions for 3 vehicles
new_df['Cluster'] = kmeans.fit_predict(new_df[['Distance_to_Location (km)', 'TimeDifference']])

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [7]:
new_df.head(10)

Order_ID Customer_Location  Package_Weight (kg) Vehicle_ID  \
0 20        21        location_u                  2.8         V1   
  41        42       location_pp                  6.9         V1   
  26        27       location_aa                  4.0         V1   
  0          1        location_a                  5.2         V1   
  11        12        location_l                  6.8         V1   
  14        15        location_o                  4.9         V1   
  5          6        location_f                  4.2         V1   
  29        30       location_dd                  5.9         V1   
  17        18        location_r                  6.2         V1   
  38        39       location_mm                  3.4         V1   

      Distance_to_Location (km)  TimeDifference  Cluster  
0 20                         10               1        0  
  41                         12               2        0  
  26                         14               2        0  
  0                          15               2        0  
  11                         19               2        2  
  14                         21               2        2  
  5                          22               2        2  
  29                         22               2        2  
  17                         24               2        1  
  38                         24               2        1

In [17]:
# Trips based on clusters, vehicle constraints for best efficiency
def tripps(df, max_weight=15, max_trips=4):
    trips = []
    current_trip = []
    current_weight = 0
    current_cluster = None

    # Sorting
    df = df.sort_values(by=['Cluster', 'Distance_to_Location (km)'])

    for _, row in df.iterrows():
        package_weight = row["Package_Weight (kg)"]
        cluster = row["Cluster"]

        # If current cluster changes new trip will start
        if current_cluster is None:
            current_cluster = cluster

        # for same cluster checking weight condition
        if current_weight + package_weight <= max_weight and cluster == current_cluster:
            current_trip.append(row)
            current_weight += package_weight
        else:
            # Save the current trip and start a new trip
            trips.append(current_trip)
            current_trip = [row]
            current_weight = package_weight
            current_cluster = cluster

        # when max trips are reached
        if len(trips) >= max_trips:
            break

    # Add any remaining deliveries to the final trip
    if current_trip and len(trips) < max_trips:
        trips.append(current_trip)

    return trips

In [18]:
vehicle_routes = {}

for vehicle_id, trip_data in new_df.groupby('Vehicle_ID'):
    trips = tripps(trip_data)

    trip_details = []
    for trip in trips:
        trip_distance = sum([a['Distance_to_Location (km)'] for a in trip])
        trip_weight = sum([b['Package_Weight (kg)'] for b in trip])
        route_sequence = [c['Distance_to_Location (km)'] for c in trip]
        orders_picked = [d['Order_ID'] for d in trip]
        trip_details.append({'route_sequence': route_sequence, 'trip_distance': trip_distance, 'trip_weight': trip_weight, 'orders_picked':orders_picked})

    # Sort trips by ascending distance
    sorted_trip_details = sorted(trip_details, key = lambda x: x['trip_distance'])
    vehicle_routes[vehicle_id] = sorted_trip_details

# Output the optimal routes for each vehicle with route details in ascending order of distance
for vehicle_id, trips in vehicle_routes.items():
    print(f"\nRoutes for {vehicle_id}:")
    for i, trip in enumerate(trips, 1):
        print(f"  Trip {i}:")
        print(f"    Locations: {trip['route_sequence']}")
        print(f"    Total Distance: {trip['trip_distance']:.2f} km")
        print(f"    Total Weight: {trip['trip_weight']:.2f} kg")
        print(f"    Total Orders: {trip['orders_picked']}")


Routes for V1:
  Trip 1:
    Locations: [15]
    Total Distance: 15.00 km
    Total Weight: 5.20 kg
    Total Orders: [1]
  Trip 2:
    Locations: [10, 12, 14]
    Total Distance: 36.00 km
    Total Weight: 13.70 kg
    Total Orders: [21, 42, 27]
  Trip 3:
    Locations: [24, 24]
    Total Distance: 48.00 km
    Total Weight: 9.60 kg
    Total Orders: [18, 39]
  Trip 4:
    Locations: [25, 25]
    Total Distance: 50.00 km
    Total Weight: 13.60 kg
    Total Orders: [3, 45]

Routes for V2:
  Trip 1:
    Locations: [28]
    Total Distance: 28.00 km
    Total Weight: 6.10 kg
    Total Orders: [32]
  Trip 2:
    Locations: [14, 15, 16]
    Total Distance: 45.00 km
    Total Weight: 14.00 kg
    Total Orders: [47, 23, 14]
  Trip 3:
    Locations: [11, 13, 13, 14]
    Total Distance: 51.00 km
    Total Weight: 13.60 kg
    Total Orders: [38, 17, 29, 11]
  Trip 4:
    Locations: [26, 28]
    Total Distance: 54.00 km
    Total Weight: 10.90 kg
    Total Orders: [20, 8]

Routes for V3:
  Trip